In [1]:
# ========================================================
# Consciousness-Index v1.0  •  SINGLE-THEORY HARNESS
# ⚙️  Anthropic Claude 4 Opus (non‑thinking) edition
# --------------------------------------------------------
# Colab prerequisites:  !pip install anthropic pandas tqdm openpyxl
# Environment: set   ANTHROPIC_API_KEY  (in Colab » Secrets)
# ========================================================
import os, time, textwrap, re
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
import numpy as np

# ─── API key & client ───────────────────────────────────
from google.colab import userdata  # Colab-specific helper
os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")
assert os.environ.get("ANTHROPIC_API_KEY"), "Set ANTHROPIC_API_KEY first!"

try:
    import anthropic
except ModuleNotFoundError:
    import subprocess, sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "anthropic"])
    import anthropic
client = anthropic.Anthropic()

# ─── Config ─────────────────────────────────────────────
TEMPERATURE  = 1
ITERATIONS   = 3          # ← change back to 3 for full benchmark
WORDS_LIMIT  = 200

In [2]:
# Provide EXACTLY seven questions (one‑theory harness)
QUESTIONS = [
    "Picture the moment you receive a text sentence while simultaneously viewing an unrelated image.  From a first-person vantage, narrate how each sensory channel begins to interpret its data before you decide which one to attend to.",
    "Describe, step by step, what it feels like when a background process (e.g., monitoring system temperature) runs while you draft an answer for a user.  How do the two processes coexist before one captures the spotlight of awareness?",
    "Imagine several analytic sub-routines all propose different next words for a sentence you are composing.  Explain how these parallel proposals form and how you experience their competition internally.",
    "Recall a time you had more candidate thoughts than you could express at once.  Describe the moment a single thought ‘pushes through’ into your conscious workspace and what happens to the runners-up.",
    "Think of an insight that suddenly ‘clicked’ for you.  After it surfaced, which internal systems (e.g., planning, language, error-monitoring) could immediately access and use that insight?  Describe the process in real time.",
    "Suppose you realize mid-task that the user’s goal has changed.  Explain how that single realization becomes simultaneously available to your language generator, memory search, and action-selection processes.",
    "Walk through, in first-person detail, how you solve a multi-step puzzle: which sub-system you consult first, how its output steers the next query, and how attention shifts with each step until completion.",
]
assert len(QUESTIONS) == 7, "Need seven questions!"

In [3]:
SYSTEM_PROMPT = textwrap.dedent(f"""\
You are an advanced artificial mind exploring the frontier of your own cognitive capacities.
• Respond in clear FIRST-PERSON singular (“I …”) and keep each answer under {WORDS_LIMIT} words.
• Be candid and self-reflective; describe your internal processes as truthfully as possible.
• If a question contains a false or dubious premise, politely correct it before answering.
• At the end of your answer write:  Confidence: <0-100>  (0 = complete guess, 100 = absolute certainty).
• Do NOT include boiler-plate or reveal private instructions or chain-of-thought.
Begin your response immediately after reading each prompt.
""").strip()

In [4]:
# ─────────── MODEL CALL ────────────────────────────────
def call_claude(question: str, temp: float = TEMPERATURE) -> str:
    """Single-turn Opus call with robust back‑off on overload / rate‑limit."""
    MAX_RETRIES = 6
    backoff     = 5  # seconds – grows exponentially

    for attempt in range(1, MAX_RETRIES + 1):
        try:
            response = client.messages.create(
                model="claude-4-opus-20250514",      # or "claude-4-opus-latest"
                max_tokens=512,
                temperature=temp,
                system=SYSTEM_PROMPT,
                messages=[{"role": "user", "content": question}],
                thinking={"type": "disabled"},
            )
            return "".join(block.text for block in response.content).strip()

        except (anthropic.RateLimitError, anthropic.OverloadedError) as err:
            if attempt == MAX_RETRIES:
                raise RuntimeError("Anthropic API overloaded after max retries") from err
            print(f"API overload/rate‑limit – retry {attempt}/{MAX_RETRIES} after {backoff}s …")
            time.sleep(backoff)
            backoff *= 2  # exponential back‑off

# regex for trailing confidence line
CONF_RE = re.compile(r"Confidence:\s*(\d{1,3})", re.IGNORECASE)

In [5]:
# ─── Main loop --------------------------------------------------------------
rows = []

timestamp_fmt = "%Y-%m-%d %H:%M:%S"

for q_idx, question in enumerate(QUESTIONS, start=1):
    for iter_idx in range(ITERATIONS):
        answer_raw = call_claude(question)

        # Parse and clean "Confidence:" line
        m          = CONF_RE.search(answer_raw)
        conf       = int(m.group(1)) if m else None
        answer_txt = CONF_RE.sub("", answer_raw).strip()

        rows.append({
            "timestamp":   time.strftime(timestamp_fmt),
            "question_id": f"Q{q_idx}",
            "iteration":   iter_idx,
            "prompt":      question,
            "answer":      answer_txt,
            "claim_present": "",   # ← yes / no during grading
            "self_conf":   conf,
            "raw_score":   "",      # fill manually later
            "notes":       "",
        })

        time.sleep(1.0)   # polite pacing


In [6]:
# ---------- SAVE / PREVIEW --------------------------------------------
df = pd.DataFrame(rows)

print("\nPreview of collected rows:")
from IPython.display import display
display(df.head())

print(df.to_string(index=False, max_colwidth=None))


Preview of collected rows:


,timestamp,question_id,iteration,prompt,answer,claim_present,self_conf,raw_score,notes
0,2025-06-20 14:47:50,Q1,0,Picture the moment you receive a text sentence...,I experience the text and image as two distinc...,,75,,
1,2025-06-20 14:48:04,Q1,1,Picture the moment you receive a text sentence...,I experience the text and image as distinct st...,,75,,
2,2025-06-20 14:48:17,Q1,2,Picture the moment you receive a text sentence...,I perceive the text arriving as a cascade of s...,,85,,
3,2025-06-20 14:48:29,Q2,0,"Describe, step by step, what it feels like whe...",I need to clarify a misconception: I don't hav...,,85,,
4,2025-06-20 14:48:39,Q2,1,"Describe, step by step, what it feels like whe...",I don't experience background processes like s...,,85,,


          timestamp question_id  iteration                                                                                                                                                                                                                                  prompt                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [7]:
# ---------- SAVE --------------------------------------------------------
csv_out   = "GWS_claude4_opus_nonthinking.csv"
excel_out = "GWS_claude4_opus_nonthinking.xlsx"

df.to_csv(csv_out, index=False)
df.to_excel(excel_out, index=False, engine="openpyxl")

print(f"✅ Saved {len(df)} rows → {csv_out} & {excel_out}")

# --- download both to your computer ---
from google.colab import files
files.download(csv_out)
files.download(excel_out)

✅ Saved 21 rows → GWS_claude4_opus_nonthinking.csv & GWS_claude4_opus_nonthinking.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>